<a href="https://colab.research.google.com/github/michaelawe01/Research/blob/main/speckle_connect_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install specklepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 9.6 MB/s eta 0:00:00
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3568 sha256=f31

In [14]:
%%writefile speckle_connect.py

import streamlit as st
import pandas as pd
from specklepy.api.wrapper import StreamWrapper
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_account_from_token
import plotly.express as px
from specklepy.api import operations
from specklepy.api.resources.current import active_user_resource

st.set_page_config(
    page_title="Speckle stram activity",
    page_icon="b"
    )

header = st.container()
input = st.container()
data = st.container()
viewer = st.container()
report = st.container()

with header:
  st.title('Michael Speckle APP')
  st.info("Specklepy exercise for creating schedules")

#Input
with input:
  st.subheader("Inputs")

  serverCol, tokenCol = st.columns([1,2])

  speckleServer = serverCol.text_input("Speckle Server", "https://app.speckle.systems")
  speckleToken = tokenCol.text_input("Speckle Token", "YOUR OWN TOKEN")


  client = SpeckleClient(host=speckleServer)
  account = get_account_from_token(speckleToken, speckleServer)
  client.authenticate_with_account(account)



  projects = client.active_user.get_projects()
  projectNames = [p.name for p in projects.items]
    # Dropdown for project selection
  pName = st.selectbox(label="Select your project", options=projectNames)

    # Find the selected project by name
  selected_project = next((p for p in projects.items if p.name == pName), None)

    # Get models related to the selected project
  models = client.model.get_models(project_id=selected_project.id)
  modelNames = [m.name for m in models.items]
    # Dropdown for model selection
  mName = st.selectbox(label="Select your model", options=modelNames)

    # Find the selected model by name
  selected_model = next((m for m in models.items if m.name == mName), None)

    # Get versions related to the selected model
  versions = client.version.get_versions(project_id=selected_project.id, model_id=selected_model.id, limit=100)


    # Display the results (optional)
  st.write("Selected Project:", selected_project)
  st.write("Selected Model:", selected_model)
  st.write("Versions:", versions)

#create a definition to convert your list to markdown
def listToMarkdown(list, column):
    list = ["- " + i + " \n" for i in list]
    list = "".join(list)
    return column.markdown(list)


def model2viewer(projects, models):
    embed_src = f"https://app.speckle.systems/projects/{projects.id}/models/{models.id}#embed=%7B%22isEnabled%22%3Atrue%7D"
    return embed_src  # Return the correct URL

with viewer:
    st.subheader("Latest version👇")
    #<iframe title="Speckle" src="https://app.speckle.systems/projects/96d2667014/models/a0db08e966#embed=%7B%22isEnabled%22%3Atrue%7D" width="600" height="400" frameborder="0"></iframe>
    #st.components.v1.iframe(src= "https://app.speckle.systems/projects/96d2667014/models/a0db08e966#embed=%7B%22isEnabled%22%3Atrue%7D", width=600, height=400)



    embed_link = model2viewer(selected_project, selected_model)

    #embed_link = model2viewer(projects, models)
    st.components.v1.iframe(src=embed_link, width=600, height=400)



#REPORT
with report:
    st.subheader("Statistics")

    projectCol, versionCol, connectorCol, contributorCol = st.columns(4)


    connectorList = [v.sourceApplication for v in versions.items]

    projectCol.metric(label="Number of Projects", value = len(projects.items))
    #st.write([p.name for p in projects.items])
    listToMarkdown([p.name for p in projects.items], projectCol)


    versionCol.metric(label="Number of versions", value = versions.totalCount)

    connectorCol.metric(label="Number of connectors", value = len(dict.fromkeys(connectorList)))
    listToMarkdown([v.sourceApplication for v in versions.items], connectorCol)

    #contributorCol.metric(label="Number of collaborators", value = len(selected_model.author))
    #st.write([selected_model.author])


#not motivated to continue as the rest of the graphs is not part of the scope of what I am learning to build. This updates will be useful for anyone looking to follow through your work.




Overwriting speckle_connect.py


In [15]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠇

In [16]:
!streamlit run /content/speckle_connect.py &>/content/logs.txt &
!curl icanhazip.com
!npx localtunnel --port 8501 &


34.74.117.98
⠙⠹⠸your url is: https://kind-onions-help.loca.lt
